In [1]:
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVR
from sklearn.pipeline import make_pipeline
import numpy as np
import pandas as pd
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")

In [2]:
cd "C://Users//Administrator//Desktop"

C:\Users\Administrator\Desktop


# Section1:Load data

In [3]:
delta_open1 = pd.read_excel('Children_pretest_open1_relative_power_band.xlsx', 'delta_open1')
theta_open1 = pd.read_excel('Children_pretest_open1_relative_power_band.xlsx', 'theta_open1')
alpha_open1 = pd.read_excel('Children_pretest_open1_relative_power_band.xlsx', 'alpha_open1') 
beta_open1 = pd.read_excel('Children_pretest_open1_relative_power_band.xlsx', 'beta_open1')

In [4]:
from pathlib import Path
Path("C://Users//Administrator//Desktop")

WindowsPath('C:/Users/Administrator/Desktop')

In [5]:
delta_open1.columns
electrodes = delta_open1.columns[1:16]
X_delta = delta_open1[electrodes]
X_theta = theta_open1[electrodes]
X_alpha =alpha_open1[electrodes]
X_beta = beta_open1[electrodes]
X = np.hstack([X_delta, X_theta, X_alpha, X_beta])
y = delta_open1['age']

In [6]:
X_train,X_test,y_train,y_test= train_test_split(X,y,test_size=0.20,stratify=y)

In [7]:
train_test_split?

# Secion2: Construct model optimized via GridSearch

In [8]:
pipe_svr=make_pipeline(StandardScaler(),SVR(kernel = "rbf"))

# Section3 Optimize SVR model via GridSearch

In [9]:
param_range_C=[1,10,100]
param_range_gamma=[0.01, 0.1]
param_grid=[{'svr__C':param_range_C, 'svr__gamma':param_range_gamma}]


gs=GridSearchCV(estimator=pipe_svr  ,
                param_grid=param_grid,
                scoring='neg_mean_absolute_error',
                cv=4)
scores=cross_val_score(gs,X_train,y_train, scoring='neg_mean_absolute_error',cv=5)


gs.fit(X_train,y_train)
y_pred = gs.predict(X_test)
a = np.vstack((y_test, y_pred))
#plt.scatter(y_pred, y_test)

# Section4 Output

In [10]:
#optimal parameters 
print(gs.best_params_)

#Mean cross-validation score of the best estimator
print("MAE in Train Phase: %.3f" % gs.best_score_)

#correlation between y_predict and y_test
print(np.corrcoef(a))

print("CV Accuracy in Train Phase: %.3f +/- %.3f" % (np.mean(scores),np.std(scores)))
print(scores)
print(gs.cv_results_)
print(np.mean(abs(y_pred - y_test)))

{'svr__C': 10, 'svr__gamma': 0.1}
MAE in Train Phase: -0.662
[[1.         0.23725342]
 [0.23725342 1.        ]]
CV Accuracy in Train Phase: -0.701 +/- 0.130
[-0.71939208 -0.48096556 -0.64711802 -0.80987388 -0.84751662]
{'mean_fit_time': array([0.00373971, 0.00324136, 0.00394523, 0.00324106, 0.00398827,
       0.00374049]), 'std_fit_time': array([4.31604818e-04, 4.31501613e-04, 7.50888897e-05, 4.31536080e-04,
       3.37174788e-07, 4.29436881e-04]), 'mean_score_time': array([0.00124705, 0.00124669, 0.00106674, 0.00174671, 0.00124788,
       0.00102633]), 'std_score_time': array([4.31467233e-04, 4.31673644e-04, 7.32582307e-05, 4.31986188e-04,
       4.31811492e-04, 4.94531444e-05]), 'param_svr__C': masked_array(data=[1, 1, 10, 10, 100, 100],
             mask=[False, False, False, False, False, False],
       fill_value='?',
            dtype=object), 'param_svr__gamma': masked_array(data=[0.01, 0.1, 0.01, 0.1, 0.01, 0.1],
             mask=[False, False, False, False, False, False],
   